# 5.4. 自定义层
- 我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
    - 研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。

## 5.4.1. 不带参数的层

In [12]:
"""
下面的CenteredLayer类要从其输入中减去均值。 要构建它，我们只需继承基础层类并实现前向传播功能。
"""
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X-X.mean()

# 让我们向该层提供一些数据，验证它是否能按预期工作。
layer=CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [18]:
# 现在，我们可以将层作为组件合并到更复杂的模型中。
net = nn.Sequential(nn.Linear(8,128),CenteredLayer())

# 作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。 由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。
Y=net(torch.rand(4,8))
Y.mean()

tensor(-5.8208e-09, grad_fn=<MeanBackward0>)

## 5.4.2. 带参数的层
以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层， 这些参数可以通过训练进行调整。 我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。 比如管理访问、初始化、共享、保存和加载模型参数。 这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

In [19]:
"""
现在，让我们实现自定义版本的全连接层。 回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。 在此实现中，我们使用修正线性单元作为激活函数。 该层需要输入参数：in_units和units，分别表示输入数和输出数。
"""
class MyLinear(nn.Module):
    def __init__(self,in_units,units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units,units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear=torch.matmul(X,self.weight.data)+self.bias.data
        return F.relu(linear)

# 实例化MyLinear类并访问其模型参数
linear=MyLinear(5,3)
linear.weight


Parameter containing:
tensor([[ 1.3495,  1.5260,  0.7970],
        [ 0.0106, -0.1521,  1.4677],
        [ 0.3302, -0.2449, -0.0254],
        [-0.4874, -0.1067, -0.2962],
        [ 0.1584,  1.0577, -0.4571]], requires_grad=True)

In [20]:
## 使用自定义层直接执行前向传播计算
linear(torch.rand(2,5))

tensor([[0.9402, 0.0000, 0.0000],
        [1.5281, 0.0000, 0.0000]])

In [21]:
# 使用自定义层构建模型，就像使用内置的全连接层一样使用自定义层。
net = nn.Sequential(MyLinear(64,8),MyLinear(8,1))
net(torch.rand(2,64))

tensor([[14.0421],
        [ 9.1927]])

## 5.4.3. 小结
- 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
- 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
- 层可以有局部参数，这些参数可以通过内置函数创建。